# 0. What for
1. Use the params values to calculate U and P which is (part of) input of MB-MDN.
2. The "the params values" generated from `GT_2_gen_uniq_params.ipynb`
# 1. Preparations
## 1.1 Global

In [49]:

ARTIFICIAL = False           # Whether to use synthetic dataset.

seed = 3
noise_pct = 0.05

# auction config
settings_NN_path = r"../data/target_datakey_all.csv"

# inferred params
if ARTIFICIAL:
    params_opitim_path =  r"../../data/SA_PT/params_artificial_v2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
else:
    params_opitim_path = "../../data/SA_PT/params_seed="+str(seed)+".csv"


unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_endprice']
LEN = 300

# output path
if ARTIFICIAL:
    filename_P = r"../../data/SA_PT/results/GT_2_artificial_SA_LEN=" +str(LEN) +"_noise="+str(noise_pct)+"_seed="+str(seed)+ ".csv"
else:
    filename_P = r"../../data/SA_PT/results/GT_2_NN_LEN=300_seed="+str(seed)+ ".csv"

import pandas as pd
from SA_for_GT2_funcs import *
from tqdm import tqdm

## 1.2 Read in data


In [50]:
data_key = pd.read_csv(settings_NN_path, encoding="utf-8")

params_all = pd.read_csv(params_opitim_path, encoding="utf-8")

LEN_KEY= data_key.shape[0]

# 2. U & P
## 2.1 Generate according to GT2

1. There is $n \leq T $
2. `U[i][0]` is initialized as 1 for convenience .
3. `U[i][j]` means：
> The probability that somebody makes the jth bid (given that j − 1 previous bids have been made)
4. `P` is the final output. `P[i]` represents the probability that auction ends at `duration=i+1`


In [51]:
col_names = []
tmp = np.arange(0,LEN)
tmp_str = [str(x) for x in tmp]
col_names.extend(tmp_str)

P_df = pd.DataFrame(columns=col_names)

P_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299


In [52]:
# solve for U & P
for i in tqdm(range(LEN_KEY)):

    # Get params
    v = float(data_key.iloc[i,3])
    d = float(data_key.iloc[i,1])
    b = float(data_key.iloc[i,2])
    params = params_all.iloc[i,:]

    alpha = params[0].item()
    labda = params[1].item()

    # d==0 suggests a fixed-price auction
    if d == 0:
        T = np.inf                                    # duration limitation
    else:
        T = np.floor((v-b)/d)                         # duration limitation

    if T >= LEN:
        U = get_U_GT2(LEN,v, d, b, alpha, labda,eps = 0.)
    elif T < LEN:
        U = [0] * (LEN + 2)                     # the prob. that someone offers a bid in t_th round
        U[0],U[1] = 1,1
        for t in range(2,len(U)):
            if(t <= T):
                U[t] = f_Equi(t, v, d, b, alpha, labda)
            else:
                U[t] = 0
            assert U[t]>=0, "U[t]<0! when t ={},and b = {},v = {}, d = {}".format(t,b,v,d)
            assert U[t]<=1, "U[t]>1! when t ={},and b = {},v = {}, d = {}".format(t,b,v,d)

    # Solve for P with length of LEN
    P = get_P(U,LEN)

    # P[i][t] = U[i][1]*U[i][2]*...*(1-U[i][t+1])
    assert len(P)==LEN,"P has wrong length (should be LEN)"


    # Concat with dataframe
    pd_tmp = pd.DataFrame(P).T
    pd_tmp.columns = col_names
    P_df = pd.concat([P_df,pd_tmp],ignore_index=True)

print("Done")


100%|██████████| 1276/1276 [00:03<00:00, 328.00it/s]

Done


# 3. save

In [53]:
filename_P

'../../data/SA_PT/results/GT_2_artificial_SA_LEN=300_noise=0.05_seed=4.csv'

In [54]:
P_df.to_csv(filename_P,header=True,index=False,encoding="utf-8")
print("DONE")

DONE
